# Day 3 - Numpy and Numerical Methods

## Agenda
1. numpy
  1. creating basic ndarrays.
  2. indexing + slicing.
  3. math. a word to the wise.
  4. broadcasting, an introduction.
  5. worked example.
2. numerical methods
  1. finding zeros of functions.
  2. scipy.minimize .
  3. simple derivatives and integrals.
  4. scipy.interpolate .
  5. project.

In [ ]:
# imports for this notebook
import numpy as np
np.set_printoptions(suppress=True) # disable scientific notation
import scipy.optimize, scipy.interpolate, scipy.integrate, scipy.misc
import time
import matplotlib.pyplot as plt

## Part 1: numpy

universally accepted as the core computing library for python. pretty much every other major library is built on top of numpy: scipy, pandas, matplotlib, xarray, Dask, scikit-learn, PyTorch, TensorFlow, seaborn, plotly....

pretty much everyone imports it like above: `import numpy as np`, that's how you'll pretty much always see it in documentation

n.b.: for numpy, it's generally best practice to not overwrite standard python functions, i.e. don't use `from numpy import [packages]`. too many shared names across libraries, e.g. the python built-in `sum()`, numpy `np.sum()`, and pandas `pd.sum()` have slightly different functionalities and under-the-hood methods; don't mix them up in your imports bc then you're not sure which library you're using when calling `sum()`.

n.b.b: numpy is a very large library. where you can, import specific modules rather than all of numpy; if you're only using `ndarray`s and their methods, then call `import numpy.ndarray`, not the full library. especially important when doing things on Midway, e.g.

### > (i) creating basic numpy arrays.

In [ ]:
a = [1,2,3,4,5] #basic python array
aa = np.array([1,2,3,4,5]) #basic np array
print(a)
print(aa)

In [ ]:
print(type(a))
print(type(aa))

In [ ]:
print(len(a))
print(len(aa))
print(aa.shape)
print(a.shape)

In [ ]:
b = [[1,2],[3,4],[5,6]]
bb = np.array(b)
print(b)
print(bb)
print(len(b))
print(len(bb))
print(bb.shape)

In [ ]:
a = np.zeros((2,2))
print(a)
print("")

b = np.ones((2,3,2))
print(b)
print("")

c = np.random.random(10)
print(c)
print("")

d = np.eye(5)
print(d)
print("")

e = np.diag([1,2,3,4])
print(e)

dtypes specify the 'number type' of the values we store in the arrays. it is a memory allocation; by default, it will be float64 (occasionally int64, depending on what is passed, like diag above)

very easy parameter to add

In [ ]:
a = np.array([1,2,3,4,5])
print(a)
b = np.array([1,2,3,4,5], dtype='float32') # dtype=np.float32 also works, if you prefer
print(b)

however! you must be careful with this, to make sure you don't overfill the memory allocations!!

In [ ]:
a = np.array([127, 128, 129], dtype='int8')
print(a)

`arange()` and `linspace()` are also commonly used methods to instantiate arrays. what's the difference? 

In [ ]:
print(np.arange(10))
print(np.arange(2,7)) # defaults to int where possible
print(np.arange(0,1,.1))
print(np.arange(0,1,.333))
print(np.arange(0,1,.249999999))

In [ ]:
print(np.linspace(1,10,1))
print(np.linspace(1,10,10)) # always defaults to float
print(np.linspace(1,10,10, dtype='int'))
print(np.linspace(0,1,5))
print(np.linspace(0,1,101))

how do we combine ndarrays?

In [ ]:
# horizontal stacking, i.e. row1 + row1, row2 + row2
a = np.array([[0,0],[1,1]]) # shape(2,2)
b = np.array([[2,2,2],[3,3,3]]) # shape(2,3)
print(a)
print("")
print(b)
print("")
print( np.hstack((a,b)) )

In [ ]:
# vertical stacking, i.e. col1 + col1, col2 + col2
bt = b.T # shape(3,2)
print(a)
print("")
print(bt)
print("")
print( np.vstack((a, bt)) )

if you don't match the dimensions correctly, you will get a moderately confusing error message:

In [ ]:
print( np.hstack((a, bt)) )

In [ ]:
# we also have np.block, to make larger arrays
A = np.ones((2, 2))
B = np.eye(2)
C = np.zeros((2, 2))
D = np.diag((-3, -4))
print(np.block([[A, B],[C,D]]))

by far the most useful of these methods is the `reshape()` function. when you are manually creating your arrays, it is often faster to create a one-dimensional matrix with all of your values, then `reshape` it into the desired dimensionality, than it is to outright create the matrix in the correct dimensionality.

In [ ]:
a = np.arange(100)
a = a.reshape((10,10))
print(a)

finally, we can read in `.csv` and `.txt` files (and other data types) directly as `ndarray`s, using the function `np.load()`. we won't do that today, but it is very useful when you're doing data analysis and have transformed a bunch of your data into `ndarray` format. it can then be saved directly as a `.npy` file, which makes reading it in at a later date dramatically faster than saving everything as `.csv` and converting to `ndarray`.

### > (ii) indexing + slicing.

In [ ]:
# standard python version
a = [[1,2,3],[4,5,6]]
print(a[0])
print(a[0][1:])
print(a[1:][1:])
print("")

# numpy
aa = np.array(a)
print(aa[0])
print(aa[0][1:])
print(aa[0,1:]) # best way to access numpy arrays
print(aa[1:,1:])

In [ ]:
# why use slicing?
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(a)
print(a.shape)
print("")

b = a[:2, 1:2]
print(b)
print(b.shape)
print("")

c = a[:2, 1]
print(c)
print(c.shape)

however! we have to be careful:

In [ ]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(a)
print("")

# let's get the first column and keep the shape
b = a[:,0:1]
print(b)
print("")

# modify this new array
b += 5
print(b)
print("")

print(a) # :(

everytime we slice and dice a numpy array, __WE ARE NOT CREATING A NEW ARRAY__. we are only creating a 'view' along a particular cross-section. be very careful when changing numpy arrays.

if you would like to create a new array, then use `copy()`:

In [ ]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(a)
print("")

# let's get the first column and keep the shape
b = a[:,0:1].copy()
print(b)
print("")

# modify this new, COPIED array
b += 5
print(b)
print("")

print(a) # :)

numpy also lets us do 'integer array indexing', which basically just means "using a numpy array to select values from a different numpy array"

In [ ]:
a = np.array([[1,2], [3, 4], [5, 6]])
b = np.array([[0,1,2],[0,1,0]]) # just for clarification, can't actually use variable b in the indexing call
print(a)
print("")
print(b)
print("")

print(a[[0,1,2],[0,1,0]]) # equivalent to: np.array( [a[0,0], a[1,1], a[2,0]] )
print("")

# When using integer array indexing, you can reuse the same
# element from the source array:
print(a[[0, 0], [1, 1]])

#### Exercise

__problem__: you are given an array `a`. use integer array indexing to successfully print out the `mutated_a` array given in the code below. 

_hint_: _you should take advantage of the 'view' capabilities of selecting elements to perform the desired mutation; create a one-dimensional array that captures the columns, and then use one of the elementary array methods to index the rows_.

In [ ]:
a = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
print(a)
print("")

### your code here ###

print(a)  # should print out mutated_a, given below

# mutated_a: 
# [[11,  2,  3],
#  [ 4,  5, 16],
#  [17,  8,  9],
#  [10, 21, 12]]

boolean array indexing is very similar to what we just did, except we just use the numerical form of booleans to do the indexing.

recall that `True = 1` and `False = 0`. so then a boolean indexing call only returns values of our array where the condition is `True`.

In [ ]:
a = np.array([[1,2],[3,4],[5,6]])
bidx = a > 3
print(bidx)
print("")

# select only the truthful elements of a
print(a[bidx]) # equivalent: a[a>3]

notice, however, that it does flatten our array. if you wish to keep the original structure of the array, it is generally better to use masked arrays. we will not go over this here, but if you're interested, take a look at the documentation of `numpy.ma` [here](https://numpy.org/doc/stable/reference/maskedarray.html). for completeness, here is a simple example.

In [ ]:
masked_a = np.ma.masked_where(a>3, a)
print(masked_a)

### > (iii) math. a word to the wise.

numpy math is pretty much always element-wise. that's kinda it, really

In [ ]:
a = np.ones(10)
b = np.arange(1,11)
print(a+b)
print(a-b)
print(a*b)
print(a/b)

we also have the built-in methods `sum()`, `diff()`, `multiply()`, and `divide()`, but numpy is written in such a way that calling the `+` operator is a shortcut to `np.sum()`, and so on. __numpy is so fundamental to python that it was allowed to overwrite the basic math operators!__

one of the few times where we may not want elementwise operations is when performing dot products. for this, we use the numpy function `dot()`:

In [ ]:
# inner product
a = np.arange(1,6)
b = np.arange(1,6)
print(a.dot(b))
print(np.dot(a,b))
print()

# matrix multiplication
a = np.random.random((3,3))
print(a)
print()
b = np.diag(np.arange(3))
print(a.dot(b))

__use built-in methods wherever possible.__ never ever write your own numpy methods unless you are 1000% certain the function doesn't already exist.

suppose we want to sum every element in an 3-dimensional numpy array. here's how we would do it manually:

In [ ]:
def sum_with_loop(a):
    a_sum = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for n in range(a.shape[2]):
                a_sum += a[i,j,n]
    return a_sum

# timing how long it takes to do this 100 times
t0 = time.time()
for i in range(100):
    a = np.random.uniform(size=(100,10,5))
    a_sum = sum_with_loop(a)
t1 = time.time()
print(f"Time: {t1 - t0} seconds")
print(f"Appx {(t1-t0)/100} seconds per sum")

compare this to the built-in method:

In [ ]:
t0 = time.time()

for i in range(100):
    a = np.random.uniform(size=(100,10,5))
    a_sum = np.sum(a)
t1 = time.time()
print(f"Time: {t1 - t0} seconds")
print(f"Appx {(t1-t0)/100} seconds per sum")

using the built-in version is approximately 100 times faster for even such a simple example. trying to implement methods like `mean()` are often 4-5 orders of magnitude slower in native python, since all of the built-in methods are written in C (a compiled language) and therefore outperform anything you could ever write in python.

in fact, let's take a look at the [documentation!](https://numpy.org/doc/stable/reference/routines.math.html)

## __BREAK__

### > (v) broadcasting, an introduction.

when we have a large array and a smaller array, we often want to use the smaller array many times on the larger, e.g. a matrix whose columns we want to add the same vector to. we could stack the vector several times to get it to the correct dimension before adding the two, but numpy has a "broadcasting mechanism" that allows us to skip this often very-time-consuming step.

in a nutshell, there are 5 rules to broadcasting:
1. if two arrays do not have the same rank, add one dimension on the left until they do (i.e. 'prepend' it)
  - e.g. if we have a.shape = (3,3,10) and b.shape = (10), then b gets recast as b.shape = (1,1,10)
2. two arrays are "compatible" in a dimension if either
  - they have the same size in that dimension (e.g. (1,3,5) and (4,3,2) are compatible in dimension 2)
  - one or both has size one in that dimension
3. arrays can be broadcast only if they are compatible in all dimensions
4. the resulting array after broadcasting has the maximum size between the two broadcasted arrays at each dimension 
  - (1,3,5) and (4,3,1) gets broadcast to (4,3,5)
5. if array 1 has size 1 in a dimension, and array 2 has size > 1 in that dimension, the broadcast array acts as if it were copied along that dimension
  - in the example for (4), the values corresponding to dimension 1 (with size 1) get copied 4 times; i.e., [3] gets broadcast to [3, 3, 3, 3]
  
i personally find it very confusing, so let's take a look at some examples.

In [ ]:
# source: https://cs231n.github.io/python-numpy-tutorial/
# Compute outer product of vectors
v = np.array([1,2,3])  # v has shape (3,)
w = np.array([4,5])    # w has shape (2,)
# To compute an outer product, we first reshape v to be a column
# vector of shape (3, 1); we can then broadcast it against w to yield
# an output of shape (3, 2), which is the outer product of v and w:
# [[ 4  5]
#  [ 8 10]
#  [12 15]]
print(np.reshape(v, (3, 1)) * w)
print("")

# Add a vector to each row of a matrix
x = np.array([[1,2,3], [4,5,6]])
# x has shape (2, 3) and v has shape (3,) so they broadcast to (2, 3),
# giving the following matrix:
# [[2 4 6]
#  [5 7 9]]
print(x + v)
print()

# Add a vector to each column of a matrix
# x has shape (2, 3) and w has shape (2,).
# If we transpose x then it has shape (3, 2) and can be broadcast
# against w to yield a result of shape (3, 2); transposing this result
# yields the final result of shape (2, 3) which is the matrix x with
# the vector w added to each column. Gives the following matrix:
# [[ 5  6  7]
#  [ 9 10 11]]
print((x.T + w).T)
print()
# Another solution is to reshape w to be a column vector of shape (2, 1);
# we can then broadcast it directly against x to produce the same
# output.
print(x + np.reshape(w, (2, 1)))
print()

# Multiply a matrix by a constant:
# x has shape (2, 3). Numpy treats scalars as arrays of shape ();
# these can be broadcast together to shape (2, 3), producing the
# following array:
# [[ 2  4  6]
#  [ 8 10 12]]
print(x * 2)

broadcasting is very important when it comes to speeding up your code, as using broadcasting allows numpy's built-in vectorization methods take over. this is especially relevant when running things on supercomputing clusters like Midway, as vectorization allows for distributing the computation over many computing nodes.

### > (v) worked example.

let's go through a problem together, to practice using numpy arrays and methods.

__problem:__ let's suppose we collected some data to try to measure some important physical property*, but we suspect there are outliers that would influence the mean. instead, report the median with a 95% confidence interval.

__further information__: we are going to be simulating the data, rather than reading in a dataset, because ~i was too lazy to find one~ simulating data is good practice. let's suppose our true value $x$ follows a normal distribution $x \sim N(\mu=4, \sigma^2=2)$, and our outliers $y$ are chi-square distributed $y \sim \chi ^2(k=6)$, and approximately 5% of our data is made up of outliers. let's also suppose we have 1000 total data points (appx 9950 'real' and 50 outliers). we will use a bootstrap to approximate the true median and 95% CI.

*this is actually the technique used by Simon Newcombe in 1878 to measure the speed of light to an accuracy of about .004% error, despite having 2 outliers in his measurements.

## __BREAK__

## Part 2: Numerical Methods

this section is not meant to be an extensive discussion of numerical methods in python; it will just be a bit of an overview of a handful of the (many) major functions and uses of the `scipy.optimize` library, plus a brief foray into `scipy.interpolate`.

a word of warning: whenever you want to break out any of the numerical methods in `scipy`, __READ THE DOCUMENTATION THOROUGHLY__. there are many very similar functions that on the surface appear to do the same thing, but under the hood and in what they accomplish, can be very different. sometimes, you'll need to switch which one you use because the one you chose just won't converge, or it crashed your computer, or there's a weird error that's not real, or etc etc. it's an art, not a science, so click through the actual documentation and check the 'see also' list at the bottom of each page. 

### > (i) finding zeros of functions.

one of the most common numerical techniques is finding the zero of a function. sometimes it's finding where our ugly little polynomial crosses the x-axis; sometimes finding the zero of a derivative; but mostly, it's for solving highly nonlinear equations, where we just move all of our terms to one side of the equals sign.

if you have a function in analytic form, the best method to use `scipy.optimize.brentq()`; it uses Richard Brent's algorithm. `brentq` requires two parameters beyond the function, which are the end points of the bracket $[a,b]$ we are finding the zero in. it requires that the signs at $a$ and $b$ are opposite, to guarantee the function crosses zero.

In [ ]:
def f(x):
    return x**3 - 6*x**2 + 11*x - 6

root = scipy.optimize.brentq(f, .5, 1.5)
print(root)

root2 = scipy.optimize.brentq(f, 1.5, 2.5)
print(root2)

root3 = scipy.optimize.brentq(f, 2.5, 10)
print(root3)

root_suprise = scipy.optimize.brentq(f, 0, 10)
print(root_suprise)

root_err = scipy.optimize.brentq(f, 0, 2.5)
print(root_err)

now let's try something like $(x^2-1) \sin(x) = 2x \cos(x)$

In [ ]:
# rewrite function to (x^2-1)sinx - 2x cosx = 0
def f(x):
    return (x**2 - 1) * np.sin(x) - 2*x*np.cos(x)

root = scipy.optimize.brentq(f, 1, 1.5)
print(root)

a more sophisticated, but considerably more likely to break, method is the `fsolve()` function. it allows us to give an approximation to the root if we would like to choose one:

In [ ]:
def f(x):
    return x**3 - 6*x**2 + 11*x - 6

root = scipy.optimize.fsolve(f, 4)
print(root)

root2 = scipy.optimize.fsolve(f, 0)
print(root2)

`fsolve` is also able to solve nonlinear system of equations, such as this one:

$x \cos(y) - 4 = 0$

$xy - y - 5 = 0$

In [ ]:
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[0] * x[1] - x[1] - 5]

root = scipy.optimize.fsolve(func, [1, 1])
print(root)

`fsolve` allows you to use multidimensional functions, as well as giving a starting estimate to one-dimensional functions (which would allow us to choose the root that brentq finds in the first example), but the algorithm is prone to failure, so use with caution.

### > (ii) `scipy.optimize.minimize`

`minimize` allows us to solve a constrained equation, which is incredibly common within economics. here's a simple example.

given our equation, $f(x) = (x-1)^2 + (y-2.5)^2 = 0$, we have the following constraints:  

$x - 2y + 2 \geq 0$

$-x - 2y + 6 \geq 0$

$-x + 2y + 2 \geq 0$

both $x$ and $y$ must be greater than 0.

In [ ]:
# recast (x,y) into np.array([x0, x1])
def f(x): 
    (x[0] - 1)**2 + (x[1] - 2.5)**2

# constraints
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2}, 
        {'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6}, 
        {'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2})
bnds = ((0, None), (0, None))

res = scipy.optimize.minimize(f, (2, 1), bounds=bnds, constraints=cons)
print(res)

this is exactly what i mean, theoretically this should just be (1.4, 1.7), but go figure. i give up. i hate numerical methods.

### > (iii) simple integration and differentiation.

the functions we are about to see only work with analytic functions.

below, the examples are: 

$ \int_{0}^4 x^2 dx $

$ \int_0^\infty e^{-x} dx $

$ \frac{d}{dx} x^3 + x^2 |(x=1) $

In [ ]:
# integration
x2 = lambda x: x**2
y, err = scipy.integrate.quad(x2, 0, 4)
print(y)

invexp = lambda x: np.exp(-x)
y, err = scipy.integrate.quad(invexp, 0, np.inf)
print(y)

In [ ]:
# differentiation
def f(x):
    return x**3 + x**2

der = scipy.misc.derivative(f, 1.0, dx=1e-6)
print(der)

### > (iv) scipy.interpolate

the `interpolate` class is considerably more powerful when it comes to research. this is a python class (like we discussed tuesday) that generates an `interpolate` object.

interpolation is a method of smoothing raw data. there are many, many, methods, but the most common is polynomial interpolation, also called 'splining'. it fits polynomials to subsets of the data, then joins them where they overlap, to form a smooth, continuous, differentiable curve that approximates the original data. naturally, the higher order the polynomial, the better the fit to the data, but having too high an order will overfit your data and ruin any meaningful interpretation you may have had. 

rule of thumb is to use cubic polynomial splines to perform interpolation.

In [ ]:
x = np.linspace(-3, 3, 50)
y = np.exp(-x**2) + 0.1 * np.random.random(size=50)
plt.plot(x, y, 'ro', ms=5)

# create the UnivariateSpline object, spl
spl = scipy.interpolate.UnivariateSpline(x, y)

# where to evaluate the spline
xs = np.linspace(-3, 3, 1000)

# why we always tune the smoothing factor
plt.plot(xs, spl(xs), 'g', lw=3)
spl.set_smoothing_factor(0.25)
plt.plot(xs, spl(xs), 'b', lw=3)
plt.title("UnivariateSpline Example")
plt.show()

In [ ]:
# using the spline to find the derivative
spl_1d = spl.derivative()
plt.plot(xs, spl_1d(xs))
plt.title("Derivative using UnivariateSpline")
plt.show()

In [ ]:
# and to find the integral
spl_1i = spl.antiderivative()
plt.plot(xs, spl_1i(xs))
plt.title("Antiderivative using UnivariateSpline")
plt.show()

many other methods exist for this class, such as finding the value of all derivatives at a single point, calculating a definite integral, finding the zeros of the spline, etc. it is an incredibly useful tool when doing equation modeling during research, and I highly recommend familiarizing yourself with it if you do such work.